# Introduction

In this notebook we'll see what are Normalizing Flows exactly and play a bit with a standard implementation. Let's import what we need. We need to have [`pytorch`](https://pytorch.org/get-started/locally/) and [`nflows`](https://github.com/bayesiains/nflows)

In [ ]:
# standard python stuff
import os
import sys
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

# stuff for torch+nflows
import torch
from torch import nn
from tqdm import tqdm
from torch.nn.modules import Module

# optimizer from torch
from torch import optim

# base Flow to construct model
from nflows.flows.base import Flow
#base distribution to use
from nflows.distributions.normal import StandardNormal
# the MADE coupling layer
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform
# this adds a RandomPermutation to add variance between layers
from nflows.transforms.permutations import RandomPermutation
# this will combine modules
from nflows.transforms.base import CompositeTransform

# Transformation rules of probability density functions

The transformation rule of pdfs for a change of variable $x=f(z)$ is

$p_{X}(x)=p_{Z}\left(f^{-1}(x)\right)|\text{det} \nabla_{x}f^{-1}\left(x\right)|$

The determinant of the Jacobian can be rewritten in terms of $f$ for ease of computation as

$p_{X}(x)=p_{Z}\left(f^{-1}(x)\right)|\text{det} \nabla_{x}f^{-1}\left(x\right)|=p_{Z}\left(f^{-1}(x)\right)|\text{det} \nabla_{z}f\left(f^{-1}(x)\right)|^{-1}$

## Example:

Show how you can transform a normally distributed $x\sim \mathcal{N}(0,1)$ to a normally distributed variable $x\sim \mathcal{N}(\mu,\sigma)$ through the change of variables $x = \mu + \sigma z$ by completing this code

In [ ]:
### an example of this
N = 10000
pdf_z = st.norm(loc=0,scale=1)
Z = pdf_z.rvs(N)
plt.hist(Z,histtype='step',density=True)
ztoplot = np.linspace(-5,5,100) # dummy variables for plotting the pdf
pdf_vals_z = pdf_z.pdf(ztoplot) # evalute pdf for plotting
plt.plot(ztoplot,pdf_vals_z)
plt.xlabel('z')
plt.ylabel('Density')

In [ ]:
mu= 1.0 # arbitrary values
sigma= 0.5 # arbitrary values
X = mu+sigma*Z
plt.hist(X,histtype='step',density=True)
xtoplot = mu+sigma*ztoplot # dummy variables for plotting the pdf
gradient_xtoplot_over_z = ... # compute the gradient
pdf_vals_x = pdf_z.pdf(ztoplot) *  ... # evaluate the new pdf using the old pdf + jacobian
plt.plot(xtoplot,pdf_vals_x)
plt.xlabel('x')
plt.ylabel('Density')


# Normalizing Flows

At its essence, Normalizing Flows are bijective functions that map a sample space to a new space where data is distributed however we chose it to. That is, if we have data $x\sim p_{X}$, we want to learn an invertible function $x = f(z,\theta)$ such that $z$ follows an base distribution easy to sample from and to evaluate. The most common choice is a normal distribution $z \sim p_{Z}\equiv \mathcal{N}(0,1)$. 

$f$ will be a learnable neural network with parameters $\theta$ and an easy to compute gradient. The loss function which $\theta$ needs to minimize is nothing more than the negative Log Likelihood obtained using the transformation rule of pdfs

$\mathcal{L}=- \sum_{x\in \mathcal{D}}\text{Ln }p_{X}(x) = - \sum_{x\in \mathcal{D}}\text{Ln }[p_{Z}(f^{-1}(x,\theta))|\text{det }\nabla_{z}f|^{-1}]$

$\mathcal{L}= \sum_{x\in \mathcal{D}}\left(-\text{Ln }[p_{Z}(f^{-1}(x,\theta))]+\text{Ln }[|\text{det }\nabla_{z}f|]\right)$

And assuming a standard normal distribution 

$\mathcal{L}= \sum_{x\in \mathcal{D}}\left(-\text{Ln }\mathcal{N}\left(f^{-1}(x,\theta);0,1\right)+\text{Ln }[|\text{det }\nabla_{z}f|]\right)$

The trick is how to chose a learnable $f$ with easy gradient (which is not a problem using the gradient chain rule with standard NNs + backpropagation) but also easily invertable to go back and forth from $x$ to $z$. 



## Example

In the previous, very simplified example, we know that a good choice of $f(z,\theta)$ is simply $f(z,\theta)=\theta_{0}+\theta_{1}z$ with inverse $f^{-1}(x,\theta)=(x-\theta_{0})/\theta_{1}$ and jacobian $|\text{det }\nabla_{z}f|=|\theta_{1}|$ (which does not depend on the evaluation on $z = (x-\theta_{0})/\theta_{1}$. We can thus simply write the loss function and do a very naive grid minimization

In [ ]:
def loss_function(theta0,theta1):
    return np.sum(-st.norm(loc=0,scale=1).logpdf((X-theta0)/theta1)+np.log(theta1))

In [ ]:
theta0vals = np.linspace(0.5,1.5,100) # substitute adequate range if you changed mu, sigma before
theta1vals = np.linspace(0.3,0.7,100) # substitute adequate range if you changed mu, sigma before
theta0vals_plot, theta1vals_plot = np.meshgrid(theta0vals,theta1vals)
# print(theta0vals_plot.shape,theta1vals_plot.shape)
loss_function_vals = np.zeros(theta0vals_plot.shape)
for ntheta1val, theta1val in enumerate(theta1vals):
    for ntheta0val, theta0val in enumerate(theta0vals):
        loss_function_vals[ntheta1val,ntheta0val]=loss_function(theta0val,theta1val)
plt.contourf(theta0vals,theta1vals,loss_function_vals,cmap='gist_heat_r')
plt.axhline(sigma)
plt.axvline(mu)
plt.colorbar()

In [ ]:
theta0min, theta1min = theta0vals_plot.flatten()[np.argmin(loss_function_vals)],theta1vals_plot.flatten()[np.argmin(loss_function_vals)]
print(theta0min,theta1min)

In [ ]:
loss_function(mu,sigma),loss_function(theta0min,theta1min) # why? likely overfitting

In [ ]:
plt.hist(X,histtype='step',density=True)
# plt.plot(xtoplot,pdf_vals_x)
# now we use the min parameters explicitly with xtoplot
pdf_vals_x_bis = st.norm(loc=0,scale=1).pdf((xtoplot-theta0min)/theta1min)/theta1min
plt.plot(xtoplot,pdf_vals_x_bis)
plt.xlabel('x')
plt.ylabel('Density')

Note that we **haven't seen the true Z during training**. The technique is aimed at learning $p_{X}(x)$. We did cheat by knowing that the simple parameterization was good enough.

# Choice of f

There are [many](https://arxiv.org/pdf/1908.09257.pdf) ways to do this, but the usual trick consists of concatenating several individual, simpler modules. That is

$z_{1} = f_{1}(z)$

$z_{i} = f_{i}(z_{i-1})$ with $i=2,...,n-1$

$x=f_{n}(z_{n-1})$

and having each individual $f_{i}$ module as a simple, invertible function whose parameters are Neural Networks. The choice of module aims to be easily invertible while allowing for as much expressivity as possible. That is, we want to be able to train while also capturing Jacobians as general as possible.

Let's talk about one possible and very popular choice, **Autoregressive flows**. Other strategies can be found in the referred paper. As always, each choice has advantages and disadvantages.

## [Masked Autoregressive flows](https://arxiv.org/pdf/1705.07057.pdf)

For inputs with dimension $K$, we parameterise $z_{i,k}$ as a function of the previous dimensions in $z_{i-1}$:

$z_{i,k}=f_{i}(z_{i-1,k},g_{i,k}(z_{i-1,1:k-1}))$

That is, a function of the same input feature at the previous step $z_{i-1,k}$ and a **conditioner** g that takes the previous input and combines all the previous features $z_{i-1,1:k-1}$. When choosing a parametric, invertible form for $f$, $g_{k}$ is a map from $z_{i-1,1:k-1}$. In the previous example, because there are no previous feature dimensions as the problem was 1D, $g_{1}$ was simply $g_{1}=[\theta_{0},\theta_{1}]$.

What is the advantage of this parameterization? That the Jacobian is triangular! One can show that

$\text{det }\nabla f_{i} = \prod_{k=1}^{K}\frac{\partial z_{i,k}}{\partial z_{i-1,k}}$


The forward flow ($z_{0}=z\rightarrow z_{n}=x$) itself can be obtained in one sweep by "masking" the features appropriately (that's why it's called Masked). "Masking" for a Neural Network means setting some features to zero before feeding the input to said Neural Network. In this case, masking allows for the conditioner to be a single set of Neural Networks, one for each parameter of $f$, each of them a function $\mathbb{R}^{K}\to \mathbb{R}$ that is evaluated for $x_{i,1:k-1}$ simply by masking or setting to zero the $k:K$ remaining entries.

The inverse function and jacobian computation ($z_{0}=z\leftarrow z_{n}=x$) is more challenging computationally speaking because we need to use recursion and compute each entry one step at a time with $z_{i-1,1}=f^{-1}_{i}(z_{i,1})$ and $z_{i-1,k}=f^{-1}_{i}(z_{i,1},g_{i,k}(z_{i-1,1:k-1}))$.

If we are more interested in the inverse (for problems such as a Variational Inference), we can use **Inverse Autoregressive Flows** (IAF) where the transformation is instead:

$z_{i,k}=f_{i}(z_{i-1,k},g_{i,k}(z_{i,1:k-1}))$

(note the different index in the conditioner) and thus the recursion needs to be applied in the forward direction $z_{0}=z\rightarrow z_{n}=x$.

The rule of thumb is: **MAF** for fast density estimation and **IAF** for fast sampling. The difference in speed is not always an issue, and as usual with ML rules of thumb can be disregarded for simple enough problems...

One can also use **Coupling flows** instead of **Autoregressive** ones. Coupling flows are equally fast in the forward and backward directions, so there is no difference between density estimation and sampling. However, they may be less flexible although very good performances can be obtained with Neural Spline Flows or Non Volume Preserving transformations. In the end, one should know what to play with and decide.


## Coupling functions

In any case, once we have defined we are using MAFs or IAFs, we need to define the coupling function $f_{i}$. This defines the parameters obtained using the conditioners $g_{i,k}$ which are Neural Networks.

A very common couplign function is [MADE](https://arxiv.org/pdf/1502.03509.pdf) where the update is using a Gaussian kernel and the conditioner models the mean and variance of the Gaussian: 

$z_{i,k} = z_{i-1,k}\text{exp }\alpha_{i}(z_{i-1,1:k-1}) + \mu_{i}(z_{i-1,1:k-1})$

$\alpha_{i}$ and $\mu_{i}$ are Neural Networks, which are evaluated on the first $k-1$ features by masking the remaining $K-(k-1)$ features.

## Example

We'll use `nflows` to implement a MAF with MADE. There are many packages, with different implementations of different flows, so I recommend you always chose based on your problem. `nflows` is not perfect but it will suffice for the examples here.

In [ ]:
# Reshape things for flows
X=X.reshape(-1,1)
print(X.shape)

In [ ]:
# Define a base distribution.
base_distribution = StandardNormal(shape=[X.shape[1]])

# Define an invertible transformation.
num_layers = 5

transforms = []
for _ in range(num_layers):
    transforms.append(MaskedAffineAutoregressiveTransform(features=X.shape[1], 
                                                          hidden_features=4,num_blocks=2))
    
    transforms.append(RandomPermutation(features=1)) # useless for 1 feature

transform = CompositeTransform(transforms)

# Combine into a flow.

flow = Flow(transform, base_distribution)
optimizer = optim.Adam(flow.parameters())

Before continuing, take some time to think about the relevant hyperparameters of the model. That is, what choices have I made. An "obvious" one is that for each initialized `MaskedAffineAutoregressiveTransform` there are two Neural Networks with `hidden_features` units per layer and `num_blocks` layers. You can look for more hyperparameters by going over the source code of the `nflows` package.

In [ ]:
# transform inputs to torch tensors
X_torch = torch.tensor(X, dtype=torch.float32)
xtoplot_torch = torch.tensor(xtoplot.reshape(-1,1),dtype=torch.float32)

The `nflows` package allows for straightforward evaluation of the likelihood with

In [ ]:
flow.log_prob(inputs=X_torch)

Let's see how things look before training the Flow. We can evaluate the likelihood using

In [ ]:
pdf_vals_x = np.exp(flow.log_prob(xtoplot_torch).detach().numpy())

And we can sample using the learned likelihood using

In [ ]:
x_sample = flow.sample(len(X)).detach().numpy()

We can see how initially everything is random and thus a poor model

In [ ]:
a,b,c = plt.hist(X,histtype='step',density=True,label='Truth',color='black')
plt.hist(x_sample,histtype='step',density=True,label='Flow',color='blue')
plt.legend(loc='upper right')
plt.plot(xtoplot,pdf_vals_x)
plt.xlabel('x')
plt.ylabel('Density')

Now, let's train

In [ ]:
#number of epochs
num_iter = 1000

for i in range(num_iter):
    optimizer.zero_grad()
    # the loss is simply - E[Log Prob] !
    loss = -flow.log_prob(inputs=X_torch).mean()
    loss.backward()
    optimizer.step()

And now we can re-evaluate

In [ ]:
pdf_vals_x = np.exp(flow.log_prob(xtoplot_torch).detach().numpy())
x_sample = flow.sample(len(X)).detach().numpy()

In [ ]:
a,b,c = plt.hist(X,histtype='step',density=True,label='Truth',color='black')
plt.hist(x_sample,histtype='step',bins=b,density=True,label='Flow',color='blue')
plt.legend(loc='upper right')
plt.plot(xtoplot,pdf_vals_x)
plt.xlabel('x')
plt.ylabel('Density')

Much better! 

You can now play with dimensions or devise some tests to quantify the agreement (like two sample tests, because this simple example is one-dimensional) and define interesting problems. As for us, we'll go to one possible application, Anomaly detection, in the next notebook